In [1]:
!pip install TensorFlow
!pip install Keras
!pip install pillow
!pip install NumPy
!pip install tqdm
!pip install jupyterlab


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
pip install graphviz

In [ ]:
!pip install Keras-Preprocessing

In [3]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install pydot-ng~=2.0.0


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception #to get pre-trained model Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer #for text tokenization
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Concatenate
from keras.models import Model, load_model
from keras.layers import Input, Dense#Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout
from tqdm.notebook import tqdm #to check loop progress
tqdm().pandas()

2024-05-13 01:35:37.765321: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


0it [00:00, ?it/s]

In [2]:
# Load the document file into memory
def load_fp(filename):
    # Open file to read
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# get all images with their captions
def img_capt(filename):
    file = load_fp(filename)
    captions = file.split("\n")
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split("\t")
        # print(f"img {img} -- caption {caption}")
        if img[:-2] not in descriptions: # -2 because we are skipping the number at the end of the image name
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

#Data cleaning function will convert all upper case alphabets to lowercase, removing punctuations and words containing numbers
def txt_clean(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
      for i,img_caption in enumerate(caps):
        img_caption.replace("-"," ")
        descp = img_caption.split()
        #uppercase to lowercase
        descp = [wrd.lower() for wrd in descp]
        #remove punctuation from each token
        descp = [wrd.translate(table) for wrd in descp]
        #remove hanging 's and a
        descp = [wrd for wrd in descp if(len(wrd)>1)] # len of `s is 2 but ' is removed in punctuation step
        #remove words containing numbers with them
        descp = [wrd for wrd in descp if(wrd.isalpha())]
        #converting back to string
        img_caption = " ".join(descp)
        captions[img][i]= img_caption
        #print(f"image {img} caption  - {captions[img][i]}")
    return captions

def txt_vocab(descriptions):
    # To build vocab of all unique words
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
        # for d in descriptions[key]:
        #     [vocab.update(d.split())]
        #     print(d.split())
    return vocab

#To save all descriptions in one file
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
      for desc in desc_list:
        lines.append(key + "\t" + desc )
        data = "\n".join(lines)
        file = open(filename,"w")
        file.write(data)
        file.close()
# Set these path according to project folder in you system
dataset_text = "Flickr8k_text"
dataset_images = "Flicker8k_Dataset"
#to prepare our text data
filename = dataset_text + "/" + "Flickr8k.token.txt"
#loading the file that contains all data
#map them into descriptions dictionary 
descriptions = img_capt(filename)
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = txt_clean(descriptions)
#to build vocabulary
vocabulary = txt_vocab(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
#saving all descriptions in one file
save_descriptions(clean_descriptions, "descriptions.txt")

Length of descriptions = 8092
Length of vocabulary =  8763


In [11]:
# Set SSL 

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [22]:
def extract_features(dirc):
        model = Xception( include_top=False, pooling='avg' )
        features = {}
        for pic in tqdm(os.listdir(dirc)):
                   file = dirc + "/" + pic
                   image = Image.open(file)
                   image = image.resize((299,299))
                   image = np.expand_dims(image, axis=0)
                   #image = preprocess_input(image)
                   image = image/127.5 # Image normalization 255/2 = 127.5
                   image = image - 1.0 # Converting it to between range -1 to 1
                   feature = model.predict(image)
                   features[pic] = feature
        return features

In [3]:
# # Run only once

# #2048 feature vector
# features = extract_features(dataset_images)
# dump(features, open("features.p","wb"))
# #to directly load the features from the pickle file.
# features = load(open("features.p","rb"))

In [4]:
#load the data
def load_photos(filename):
    file = load_fp(filename)
    photos = file.split("\n")[:-1]
    return photos
def load_clean_descriptions(filename, photos):
    #loading clean_descriptions
    file = load_fp(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<2: # To skip img desc pair with no desc since len of list will be just 1 then. But it should be <2  not <1
            continue
        image, image_caption = words[0], words[1:]
        if image in photos: 
            if image not in descriptions:
                descriptions[image] = []
            desc = f' {" ".join(image_caption)} '
            # desc = ' ' + " ".join(image_caption) + ' '
            # print(desc)
            descriptions[image].append(desc)
    print(descriptions["1000268201_693b08cb0e.jpg"])            
    return descriptions
def load_features(photos):
    #loading all featuresq
    all_features = load(open("features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features
filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"
#train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

[' child in pink dress is climbing up set of stairs in an entry way ', ' girl going into wooden building ', ' little girl climbing into wooden playhouse ', ' little girl climbing the stairs to her playhouse ', ' little girl in pink dress going into wooden cabin ']


In [5]:
#convert dictionary to clear list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
       [all_desc.append(d) for d in descriptions[key]]
    # print(f"Length of all_desc {len(all_desc)}")    
    return all_desc
#creating tokenizer class
#this will vectorise text corpus
#each integer will represent token in dictionary
from tensorflow.keras.preprocessing.text import Tokenizer
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer
# give each word an index, and store that into tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))
# print(tokenizer.word_index)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size) #The size of our vocabulary is 7577 words.
#calculate maximum length of descriptions to decide the model structure parameters.
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)
max_length = max_length(descriptions)
print(max_length) #Max_length of description is 32

7577
32


In [33]:
seq = tokenizer.texts_to_sequences([' child in pink dress is climbing up set of stairs in an entry way '])[0]
x_1, x_2, y = list(), list(), list()
for i in range(1, len(seq)):
    # divide into input and output pair
    in_seq, out_seq = seq[:i], seq[i]
    # pad input sequence
    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
    # encode output sequence
    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
    # store
    x_1.append("feature")
    x_2.append(in_seq)
    y.append(out_seq)
x_1, x_2, y = np.array(x_1), np.array(x_2), np.array(y)
print(f"{x_1}\n\n{x_2}\n\n {y}")
print(x_1.shape, x_2.shape, y.shape)

['feature' 'feature' 'feature' 'feature' 'feature' 'feature' 'feature'
 'feature' 'feature' 'feature' 'feature' 'feature' 'feature']

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0   40]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0   40    1]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0   40    1   85]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
    40    1   85  167]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0   40
     1   85  167    4]
 [   0    0    0    0    0    0

In [24]:
#data generator, used by model.fit_generator()
def data_generator(descriptions, features, tokenizer, max_length):
    while True:
        for key, description_list in descriptions.items():
            #retrieve photo features
            feature = features[key][0]
            inp_image, inp_seq, op_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield ((inp_image, inp_seq), op_word)
def create_sequences(tokenizer, max_length, desc_list, feature):
    x_1, x_2, y = list(), list(), list()
    # move through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # divide one sequence into various X,y pairs
        for i in range(1, len(seq)):
            # divide into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            x_1.append(feature)
            x_2.append(in_seq)
            y.append(out_seq)
    return np.array(x_1), np.array(x_2), np.array(y)
#To check the shape of the input and output for your model
[a,b],c = next(data_generator(descriptions, train_features, tokenizer, max_length))
print(a.shape, b.shape, c.shape)
#((47, 2048), (47, 32), (47, 7577))

(37, 2048) (37, 32) (37, 7577)


In [15]:
from keras.utils import plot_model
# define the captioning model
def define_model(vocab_size, max_length):
    # features from the CNN model compressed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # Merging both models
    decoder1 = Concatenate()([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # merge it [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [ ]:
# train our model
print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)
model = define_model(vocab_size, max_length)
print(f"model {model}")
epochs = 10
steps = len(train_descriptions)
# creating a directory named models to save our models
# os.mkdir("models")
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    #model.fit([x1,x2],y)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save(f"models/model_{i}.h5")

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000
Vocabulary Size: 7577
Description Length:  32


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 32, 256)   │  1,939,712 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 2048)      │          0 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 32, 256)   │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 32)        │          0 │ input_layer_5[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 256)       │    524,544 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 256)       │    525,312 │ dropout_5[0][0],  │
│                     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 512)       │          0 │ dense_6[0][0],    │
│ (Concatenate)       │                   │            │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 256)       │    131,328 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 7577)      │  1,947,289 │ dense_7[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,068,185 (19.33 MB)

 Trainable params: 5,068,185 (19.33 MB)

 Non-trainable params: 0 (0.00 B)

None
You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.
model <Functional name=functional_5, built=True>
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 765s 127ms/step - loss: 5.4091


6000/6000 ━━━━━━━━━━━━━━━━━━━━ 953s 159ms/step - loss: 4.0154


6000/6000 ━━━━━━━━━━━━━━━━━━━━ 1084s 181ms/step - loss: 3.6392


1438/6000 ━━━━━━━━━━━━━━━━━━━━ 20:23 268ms/step - loss: 3.4596